In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("leetcode_questions.csv")

# Check available columns
print("Available columns:", df.columns.tolist())
print("Dataset shape:", df.shape)

# Create complexity from AcceptanceRate (lower acceptance = higher complexity)
df['Complexity'] = 100 - df['AcceptanceRate']

# Show topics from Topics column
topics = df['Topics'].str.split(', ').explode().unique()
print("\nAvailable topics:")
for i, topic in enumerate(topics[:10], 1):  # Show first 10 topics
    print(f"{i}. {topic}")

In [ ]:
# Adaptive system
class AdaptiveLeetCode:
    def __init__(self, dataframe):
        self.df = dataframe
        self.level = 50
        self.asked = set()
        self.correct = 0
        self.total = 0
        
    def get_question(self):
        available = self.df[~self.df.index.isin(self.asked)]
        if available.empty:
            return None
        suitable = available[
            (available['Complexity'] >= self.level - 15) & 
            (available['Complexity'] <= self.level + 15)
        ]
        if suitable.empty:
            suitable = available
        return suitable.sample(n=1).iloc[0]
    
    def answer(self, correct):
        self.total += 1
        if correct:
            self.correct += 1
            self.level = min(100, self.level + 8)
        else:
            self.level = max(10, self.level - 12)
    
    def stats(self):
        accuracy = (self.correct / max(1, self.total)) * 100
        return self.level, accuracy, self.correct, self.total

In [ ]:
# Run session
system = AdaptiveLeetCode(df)
print(f"Starting level: {system.level}")

for i in range(1, 6):
    q = system.get_question()
    if q is None:
        break
    
    system.asked.add(q.name)
    
    print(f"\nQ{i}: {q['Title']}")
    print(f"Difficulty: {q['Difficulty']}")
    print(f"Topics: {q['Topics']}")
    print(f"Complexity: {q['Complexity']:.1f}%")
    
    ans = input("Solved correctly? (y/n): ").lower() == 'y'
    system.answer(ans)
    
    level, acc, correct, total = system.stats()
    print(f"Level: {level:.1f}, Accuracy: {acc:.1f}%")

print(f"\nFinal: Level {system.level:.1f}, {system.correct}/{system.total} correct")